In [34]:
import numpy as np

In [35]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [36]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [37]:
def svd_whitening(images,epsilon):
    #images is NxD matrix N: number of data and D: dimenson of image
    #epsilon is very small number that prevents divding by zero
    images = images - np.mean(images,axis=0)
    cov = np.dot(images.T,images)/images.shape[0]
    U,S,V = np.linalg.svd(cov)
    images_Rot = np.dot(images,U)
    images_white = images_Rot/np.sqrt(S+epsilon)
    return images_white

In [38]:
def display_images(images,display_number):
    image_size = images.shape
    print(image_size)
    number_to_show = np.random.randint(1,image_size[0]+1,size=display_number)
    print(number_to_show)
    
    for i in number_to_show:
        pixels = images[i,:]
        pixels = np.array(pixels)
        pixels = pixels.reshape((28,28))
        
        plt.title(mnist.test.labels[i])
        plt.imshow(pixels,cmap='gray')
        plt.show()

In [39]:
def display_ternsorflow_variables(envs):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(sess.run(envs))

In [40]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))

In [41]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100

In [42]:
x = tf.placeholder("float32",[None,784])
y = tf.placeholder("float32",[None,10])

In [43]:
W = init_weights([784,10])
b = init_weights([10])

In [51]:
hypothesis = tf.nn.softmax(tf.matmul(x,W)+b)
epsilon = tf.constant(0.00001)
activation = hypothesis+epsilon

In [52]:
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(activation),reduction_indices=1))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [64]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            #whitened_batch = svd_whitening(batch_xs,1e-5)
            sess.run(optimizer,feed_dict={x:batch_xs, y:batch_ys})
            avg_cost += sess.run(cost, feed_dict={x:batch_xs, y:batch_ys})/total_batch
        
        print('epoch:',epoch+1,'cost=',avg_cost)
    print('opt finished')
    
    correction_prediction = tf.equal(tf.argmax(activation,1),tf.argmax(y,1))

    accuracy = tf.reduce_mean(tf.cast(correction_prediction,"float"))

    print(accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

epoch: 1 cost= 0.335440001908
epoch: 2 cost= 0.280434470962
epoch: 3 cost= 0.270825591372
epoch: 4 cost= 0.261908729469
epoch: 5 cost= 0.255700813464
epoch: 6 cost= 0.2534448709
epoch: 7 cost= 0.25544003003
epoch: 8 cost= 0.249839808413
epoch: 9 cost= 0.251918815544
epoch: 10 cost= 0.247253869806
epoch: 11 cost= 0.254164400107
epoch: 12 cost= 0.242071551884
epoch: 13 cost= 0.239325527952
epoch: 14 cost= 0.244899876626
epoch: 15 cost= 0.24552032426
epoch: 16 cost= 0.23588534893
epoch: 17 cost= 0.241261949336
epoch: 18 cost= 0.244216732986
epoch: 19 cost= 0.237653615617
epoch: 20 cost= 0.235924265899
epoch: 21 cost= 0.241560477235
epoch: 22 cost= 0.230606254624
epoch: 23 cost= 0.247159188844
epoch: 24 cost= 0.236996485659
epoch: 25 cost= 0.234835868986
opt finished
0.9191


In [62]:
batch_xs,batch_ys = mnist.train.next_batch(batch_size)
batch_xs.shape

(100, 784)